In [1]:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)


myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

In [2]:
import $ivy.`edu.holycross.shot::ohco2:10.18.2`
import $ivy.`edu.holycross.shot.cite::xcite:4.2.0`
import $ivy.`edu.holycross.shot::midvalidator:10.0.0`
import $ivy.`edu.holycross.shot::latincorpus:2.2.2`
import $ivy.`edu.holycross.shot::latphone:2.7.2`
// It would be nice to visualize, so let's use the
// plotly library with ammonite sh:
// Make plotly libraries available to this notebook:
import $ivy.`org.plotly-scala::plotly-almond:0.7.1`

Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/ohco2_2.12/10.18.2/ohco2_2.12-10.18.2.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/ohco2_2.12/10.18.2/ohco2_2.12-10.18.2.pom.sha1
Downloaded https://dl.bintray.com/neelsmith/maven/edu/holycross/shot/ohco2_2.12/10.18.2/ohco2_2.12-10.18.2.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/scala-library/2.12.4/scala-library-2.12.4.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/cex_2.12/6.3.3/cex_2.12-6.3.3.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/seqcomp_2.12/1.0.0/seqcomp_2.12-1.0.0.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/cite/xcite_2.12/4.1.1/xcite_2.12-4.1.1.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/cite/xcite_2.12/4.1.1/xcite_2.12-4.1.1.pom.sha1
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/cex_2.12/6.3.3/cex_2.12-6.3.3.pom.sha1
Downloaded https://repo1.maven.org/maven2/org/scala-lang/modules

import $ivy.$                                  

import $ivy.$                                     

import $ivy.$                                        

import $ivy.$                                      

import $ivy.$                                   
// It would be nice to visualize, so let's use the
// plotly library with ammonite sh:
// Make plotly libraries available to this notebook:

import $ivy.$                                      

In [3]:

// Import plotly libraries, and set display defaults suggested for use in Jupyter NBs:
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)


import edu.holycross.shot.cite._
import edu.holycross.shot.ohco2._
import edu.holycross.shot.latin._
import edu.holycross.shot.mid.validator._
import edu.holycross.shot.latincorpus._

import scala.io.Source


import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

import edu.holycross.shot.cite._

import edu.holycross.shot.ohco2._

import edu.holycross.shot.latin._

import edu.holycross.shot.mid.validator._

import edu.holycross.shot.latincorpus._


import scala.io.Source


In [4]:
val hyginusUrl = "https://raw.githubusercontent.com/neelsmith/hctexts/master/cex/hyginus.cex"
val corpus = CorpusSource.fromUrl(hyginusUrl, cexHeader = true)


hyginusUrl: String = "https://raw.githubusercontent.com/neelsmith/hctexts/master/cex/hyginus.cex"
corpus: Corpus = Corpus(
  Vector(
...

In [5]:
val hyginusFstUrl = "https://raw.githubusercontent.com/neelsmith/hctexts/master/parser-output/hyginus/hyginus-parses.txt"
val fstOutput = Source.fromURL(hyginusFstUrl).getLines.toVector

hyginusFstUrl: String = "https://raw.githubusercontent.com/neelsmith/hctexts/master/parser-output/hyginus/hyginus-parses.txt"
fstOutput: Vector[String] = Vector(
  "> et",
...

In [6]:
// This step can take a moment, too, as it attempts to associate a
// morphological analysis with every lexical token.
val lhyg = LatinCorpus.fromFstLines(
  corpus,
  Latin23Alphabet,
  fstOutput,
  strict = false
)

2020-03-07 12:36:30.879Z debug [LatinCorpus] Computing analyses with FstReader  - (LatinCorpus.scala:247)
2020-03-07 12:36:31.812Z debug [LatinCorpus] Done computing analyses.  - (LatinCorpus.scala:249)
2020-03-07 12:36:31.813Z debug [LatinCorpus] Now generate corpus from analyses.  - (LatinCorpus.scala:250)
2020-03-07 12:36:31.814Z debug [LatinCorpus] Creating tokenizable corpus...  - (LatinCorpus.scala:198)
2020-03-07 12:36:31.818Z debug [LatinCorpus] Done creating tokenizable corpus.  - (LatinCorpus.scala:200)
2020-03-07 12:36:35.678Z debug [LatinCorpus] Cycle through 33220 tokenizable tokens.  - (LatinCorpus.scala:201)
2020-03-07 12:36:35.686Z  warn [LatinCorpus] 
Failed on token category opt None.  Citable node was 
	CitableNode(urn:cts:latinLit:stoa1263.stoa001.hc_tkns:t.1.4,VOLGO)  - (LatinCorpus.scala:225)
2020-03-07 12:36:35.687Z  warn [LatinCorpus] 
Failed on token category opt None.  Citable node was 
	CitableNode(urn:cts:latinLit:stoa1263.stoa001.hc_tkns:t.1.5,FABVLAE)  - (

lhyg: LatinCorpus = LatinCorpus(
  Vector(
...

In [8]:
def posIdCounts(tkns: Vector[LatinToken], lexemeHistogram: edu.holycross.shot.histoutils.Histogram[String]): Vector[(String, String, Int)] = {
  tkns.map(a =>
  {
    val id = a.analyses(0).lemmaId
    val pos = a.analyses(0).posLabel
   (id, pos,lexemeHistogram.countForItem(id))
 }).distinct
}
def posCountMap(tkns: Vector[LatinToken], lexemeHistogram: edu.holycross.shot.histoutils.Histogram[String] ): Map[String, Vector[(String, String, Int)]] = {
  posIdCounts(tkns, lexemeHistogram).groupBy{ case (id,pos,count) => pos }
}



defined function posIdCounts
defined function posCountMap

In [9]:
def posOccurrences(tkns: Vector[LatinToken], lexemeHistogram: edu.holycross.shot.histoutils.Histogram[String]): Vector[(String, Int)] = {
  posCountMap(tkns, lexemeHistogram).toVector.map{ case (pos,v) => (pos, v.map(_._3).sum)}
}

def posLexemeCounts(tkns: Vector[LatinToken], lexemeHistogram: edu.holycross.shot.histoutils.Histogram[String]) : Vector[(String, Int)]= {
   posCountMap(tkns, lexemeHistogram).toVector.map{ case (pos,v) => (pos, v.size) }
}

def lexemeDensityByPos(tkns: Vector[LatinToken], lexemeHistogram: edu.holycross.shot.histoutils.Histogram[String]) : Vector[Double]= {
  val occurrenceCounts = posOccurrences(tkns, lexemeHistogram).sortBy( _._1).map(_._2)
  val lexemeCounts = posLexemeCounts(tkns, lexemeHistogram).sortBy( _._1).map(_._2)
  lexemeCounts.zip(occurrenceCounts).map {
    case (lex,occ) => occ / lex.toDouble
  }
}

defined function posOccurrences
defined function posLexemeCounts
defined function lexemeDensityByPos

In [10]:
// Now Hyginus:
val hygHist = lhyg.lexemeHistogram

hygHist: edu.holycross.shot.histoutils.Histogram[String] = Histogram(
  Vector(
...

In [11]:
val posList = posLexemeCounts(lhyg.analyzed, hygHist).sortBy( _._1).map(_._1)


posList: Vector[String] = Vector(
  "adjective",
...

In [12]:
val hygPosOccurrences = posOccurrences(lhyg.analyzed,hygHist).sortBy( _._1).map(_._2)
val hygLexemeCounts = posLexemeCounts(lhyg.analyzed,hygHist).sortBy( _._1).map(_._2)


hygPosOccurrences: Vector[Int] = Vector(
  1001,
...
hygLexemeCounts: Vector[Int] = Vector(92, 57, 15, 66, 78, 341, 175, 14, 21, 264)

In [13]:
val occurrencesBar =  Bar(x = posList, y = hygPosOccurrences, name = "Occurrences")
val lexemesBar =  Bar(x = posList, y = hygLexemeCounts, name = "Number of lexemes")



occurrencesBar: Bar = Bar(
  Strings(
...
lexemesBar: Bar = Bar(
  Strings(
...

In [14]:
val jointLayout = Layout(
  barmode = BarMode.Group,
    xaxis = Axis(
        title = "Analytical type",
        showgrid = false,
        zeroline = false
    ),
    yaxis = Axis(
        title = "Counts"
    )
)

val jointData = Seq(
  occurrencesBar,
  lexemesBar
)

jointLayout: Layout = Layout(
  None,
...
jointData: Seq[Bar] = List(
  Bar(
...

In [15]:
plot(jointData, jointLayout)

res14: String = "plot-8d6720bb-e9ef-4e2e-9d92-efc71b8ae4e8"

In [16]:
val density = lexemeDensityByPos(lhyg.analyzed, hygHist)

density: Vector[Double] = Vector(
  10.880434782608695,
...

In [17]:
val densityPlot = Vector(
  Bar(x = posList, y = density)
)
val densityLayout = Layout(
  title = "Average frequency of individual lexemes",
  xaxis = Axis(
      title = "Analytical type",
      showgrid = false,
      zeroline = false
  ),
  yaxis = Axis(
      title = "Ratio of lexemes to occurrences"
  )
)

densityPlot: Vector[Bar] = Vector(
  Bar(
...
densityLayout: Layout = Layout(
  Some("Average frequency of individual lexemes"),
...

In [18]:
plot(densityPlot, densityLayout)

res17: String = "plot-50ecde1f-f728-4d30-92fc-3947307dab8d"